In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
minor_s = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장비주력_stand.csv', index_col=0)
minor_test = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240318상장비주력test_stand.csv', index_col=0)

##### train_df, test_df 설정

In [ ]:
train_df = minor_s[['유보액대비율', '총자본투자효율', '지급이자율',
 '당좌자산회전률', '자본', '매입채무회전률','매입채무회전기간',
 '자기자본순이익률', '매출채권회전률', '자본분배율', '유연탄가격',
 '타인자본회전률','유보율','부실']]

In [ ]:
test_df = minor_test[['유보액대비율', '총자본투자효율', '지급이자율',
 '당좌자산회전률', '자본', '매입채무회전률','매입채무회전기간',
 '자기자본순이익률', '매출채권회전률', '자본분배율', '유연탄가격',
 '타인자본회전률','유보율','부실']]

In [ ]:
train_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

In [ ]:
x_train = train_df.drop("부실",axis=1)
y_train = train_df["부실"]
x_test =test_df.drop("부실",axis=1)
y_test = test_df["부실"]

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler

# # RandomUnderSampler 객체 생성
# under = RandomUnderSampler(random_state=4, sampling_strategy=0.1)

# # RandomUnderSampler를 사용하여 샘플링된 특징 및 타겟 데이터 생성
# x_resampled, y_resampled = under.fit_resample(x_train, y_train)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
x_resampled, y_resampled = RandomOverSampler(random_state=4 ).fit_resample(x_train,y_train)

y_resampled.value_counts()

0.0    3431
1.0    3431
Name: 부실, dtype: int64

In [ ]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    pr_score = average_precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))


##### 1. RandomForest

In [ ]:
# 랜덤포레스트 구현
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=52)

rf_clf.fit(x_resampled, y_resampled)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)

In [ ]:
print("랜덤포레스트")
get_clf_eval(y_test, rf_pred)

랜덤포레스트
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


##### 하이퍼파라미터 조정 & 임계값 설정한 뒤 모델링

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# RandomForestClassifier 모델 생성
rf_clf = RandomForestClassifier()

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝과 교차 검증 수행
grid_search = GridSearchCV(rf_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_rf_clf = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
rf_pred = best_rf_clf.predict(x_test)

In [ ]:
# 임계값 설정
threshold = 0.1
rf_pred = (rf_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("랜덤포레스트")
get_clf_eval(y_test, rf_pred)

랜덤포레스트
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


##### 2. LogisticRegression

In [ ]:
# 로지스틱 회귀 모델
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(x_resampled, y_resampled)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)

In [ ]:
print("로지스틱")
get_clf_eval(y_test, lr_pred)

로지스틱
오차행렬
[[1119   44]
 [   0    5]]
정확도: 0.9623, 정밀도 : 0.1020, 재현율:1.0000,F1 스코어:0.1852
ROC 스코어: 0.9811, PR score : 0.1020


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'C': [0.001, 0.01, 0.1, 0.2, 0.3, 0.5, 1.0, 10.0],
    'penalty': ['l1', 'l2']
}

# 로지스틱 회귀 모델 생성
logistic_clf = LogisticRegression()

# 그리드서치 수행
grid_search = GridSearchCV(logistic_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_logistic_clf = grid_search.best_estimator_

# 예측 확률 계산
logistic_pred_proba = best_logistic_clf.predict_proba(x_test)

In [ ]:
# 임계값 설정 (예시로 0.5 사용)
threshold = 0.01
logistic_pred = (logistic_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("로지스틱")
get_clf_eval(y_test, logistic_pred)

로지스틱
오차행렬
[[908 255]
 [  0   5]]
정확도: 0.7817, 정밀도 : 0.0192, 재현율:1.0000,F1 스코어:0.0377
ROC 스코어: 0.8904, PR score : 0.0192


##### 3. SVM

In [ ]:
# SVM 모형 구현
from sklearn import svm

svm_clf = svm.SVC(kernel='linear', probability=True)  # 선형 SVM 모델 생성
svm_clf.fit(x_resampled, y_resampled)  # 훈련 데이터를 이용하여 모델 훈련
# 테스트 데이터를 사용하여 예측
svm_pred = svm_clf.predict(x_test)
svm_pred_proba = svm_clf.predict_proba(x_test)

In [ ]:
print("SVM")
get_clf_eval(y_test, svm_pred)

SVM
오차행렬
[[1129   34]
 [   1    4]]
정확도: 0.9700, 정밀도 : 0.1053, 재현율:0.8000,F1 스코어:0.1860
ROC 스코어: 0.8854, PR score : 0.0851


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}
svm_clf = SVC(probability=True)
random_search = RandomizedSearchCV(svm_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_svm_clf = random_search.best_estimator_

# 예측 확률 계산
svm_pred_proba = best_svm_clf.predict_proba(x_test)

In [ ]:
# 임계값 설정 (예시로 0.5 사용)
threshold = 0.1
svm_pred = (svm_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("SVM")
get_clf_eval(y_test, svm_pred)

SVM
오차행렬
[[1143   20]
 [   3    2]]
정확도: 0.9803, 정밀도 : 0.0909, 재현율:0.4000,F1 스코어:0.1481
ROC 스코어: 0.6914, PR score : 0.0389


##### 4. KNN

In [ ]:
# KNN 모형 구현
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(x_resampled, y_resampled)
knn_pred = knn_clf.predict(x_test)
knn_pred_proba = knn_clf.predict_proba(x_test)

In [ ]:
print("knn")
get_clf_eval(y_test, knn_pred)

knn
오차행렬
[[1128   35]
 [   3    2]]
정확도: 0.9675, 정밀도 : 0.0541, 재현율:0.4000,F1 스코어:0.0952
ROC 스코어: 0.6850, PR score : 0.0242


In [ ]:
# RandomizedSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]  # 1은 맨해튼 거리, 2는 유클리디언 거리
}
knn_clf = KNeighborsClassifier()
random_search = RandomizedSearchCV(knn_clf, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_knn_clf = random_search.best_estimator_

# 예측 확률 계산
knn_pred_proba = best_knn_clf.predict_proba(x_test)

In [ ]:
# 임계값 설정
threshold = 0.2
knn_pred = (knn_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("KNN")
get_clf_eval(y_test, knn_pred)

KNN
오차행렬
[[1136   27]
 [   4    1]]
정확도: 0.9735, 정밀도 : 0.0357, 재현율:0.2000,F1 스코어:0.0606
ROC 스코어: 0.5884, PR score : 0.0106


##### 5. LGBM

In [ ]:
# LGBM 모형 구현
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier()
lgb_model.fit(x_resampled, y_resampled)
# 테스트 데이터를 사용하여 예측
lgb_pred = lgb_model.predict(x_test)
lgb_pred_proba = lgb_model.predict_proba(x_test)

[LightGBM] [Info] Number of positive: 3431, number of negative: 3431
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001249 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3066
[LightGBM] [Info] Number of data points in the train set: 6862, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [ ]:
print("LGBM")
get_clf_eval(y_test, lgb_pred)

LGBM
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5]
}

# LGBMClassifier 모델 생성
lgbm_clf = LGBMClassifier()

# 그리드서치 수행
grid_search = GridSearchCV(lgbm_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_lgbm_clf = grid_search.best_estimator_

# 예측 확률 계산
lgbm_pred = best_lgbm_clf.predict(x_test)
lgbm_pred_proba = best_lgbm_clf.predict_proba(x_test)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [ ]:
# 임계값 설정
threshold = 0.3
lgb_pred = (lgb_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("LGBM")
get_clf_eval(y_test, lgb_pred)

LGBM
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


##### 6. xgboost

In [ ]:
# xgboost 모형 구현
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance

xgb_wrapper = XGBClassifier()

xgb_wrapper.fit(x_resampled, y_resampled, verbose = True)


xgb_pred = xgb_wrapper.predict(x_test)
xgb_pred_proba = xgb_wrapper.predict_proba(x_test)

In [ ]:
print("xgb")
get_clf_eval(y_test, xgb_pred)

xgb
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5]
}

# XGBClassifier 모델 생성
xgb_clf = XGBClassifier()

# 그리드서치 수행
grid_search = GridSearchCV(xgb_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_xgb_clf = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
xgb_pred = best_xgb_clf.predict(x_test)
xgb_pred_proba = best_xgb_clf.predict_proba(x_test)

In [ ]:
# 임계값 설정
threshold = 0.2
xgb_pred = (xgb_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("xgboost")
get_clf_eval(y_test, lgb_pred)

xgboost
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


#### 7. catboost

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier()
catboost_model.fit(x_resampled, y_resampled)
# 테스트 데이터를 사용하여 예측
catboost_pred = catboost_model.predict(x_test)
catboost_pred_proba = catboost_model.predict_proba(x_test)

Learning rate set to 0.023448
0:	learn: 0.6453004	total: 6.69ms	remaining: 6.69s
1:	learn: 0.6051456	total: 12.6ms	remaining: 6.27s
2:	learn: 0.5696390	total: 17.9ms	remaining: 5.96s
3:	learn: 0.5323364	total: 23.2ms	remaining: 5.78s
4:	learn: 0.5003691	total: 28.7ms	remaining: 5.7s
5:	learn: 0.4720252	total: 34.1ms	remaining: 5.65s
6:	learn: 0.4458052	total: 39.5ms	remaining: 5.6s
7:	learn: 0.4183371	total: 45ms	remaining: 5.58s
8:	learn: 0.3970178	total: 51.5ms	remaining: 5.67s
9:	learn: 0.3786603	total: 56.9ms	remaining: 5.64s
10:	learn: 0.3583645	total: 62.7ms	remaining: 5.64s
11:	learn: 0.3398555	total: 68.1ms	remaining: 5.61s
12:	learn: 0.3263700	total: 73.5ms	remaining: 5.58s
13:	learn: 0.3116138	total: 78.9ms	remaining: 5.56s
14:	learn: 0.2976417	total: 84.3ms	remaining: 5.54s
15:	learn: 0.2852332	total: 91.8ms	remaining: 5.65s
16:	learn: 0.2713522	total: 97.6ms	remaining: 5.64s
17:	learn: 0.2606646	total: 103ms	remaining: 5.63s
18:	learn: 0.2497275	total: 109ms	remaining: 5.63

In [ ]:
print("catboost")
get_clf_eval(y_test, catboost_pred)

catboost
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043


In [ ]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

# 그리드서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [1, 3, 5]
}

# CatBoostClassifier 모델 생성
catboost_clf = CatBoostClassifier()

# 그리드서치 수행
grid_search = GridSearchCV(catboost_clf, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_resampled, y_resampled)

# 최적의 모델과 하이퍼파라미터 가져오기
best_catboost_clf = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 수행
catboost_pred = best_catboost_clf.predict(x_test)
catboost_pred_proba = best_catboost_clf.predict_proba(x_test)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0:	learn: 0.6739424	total: 7.76ms	remaining: 2.32s
1:	learn: 0.6575022	total: 15.6ms	remaining: 2.33s
2:	learn: 0.6392037	total: 31.9ms	remaining: 3.16s
3:	learn: 0.6230256	total: 42ms	remaining: 3.11s
4:	learn: 0.6054782	total: 50.2ms	remaining: 2.96s
5:	learn: 0.5888537	total: 58ms	remaining: 2.84s
6:	learn: 0.5740046	total: 65.9ms	remaining: 2.76s
7:	learn: 0.5602791	total: 76.5ms	remaining: 2.79s
8:	learn: 0.5449650	total: 84.1ms	remaining: 2.72s
9:	learn: 0.5316186	total: 92.1ms	remaining: 2.67s
10:	learn: 0.5179684	total: 99.6ms	remaining: 2.62s
11:	learn: 0.5049101	total: 109ms	remaining: 2.62s
12:	learn: 0.4927698	total: 117ms	remaining: 2.59s
13:	learn: 0.4799146	total: 132ms	remaining: 2.7s
14:	learn: 0.4685004	total: 143ms	remaining: 2.72s
15:	learn: 0.4585712	total: 151ms	remaining: 2.68s
16:	learn: 0.4473493	total: 159ms	remaining: 2.64s
17:	learn: 0.4372590	total: 166ms	remaining: 2.6s
18:	learn: 0.4278605	total: 175ms	remaining: 2.58s


In [ ]:
# 임계값 설정
threshold = 0.2
catboost_pred = (catboost_pred_proba[:, 1] > threshold).astype(int)

# 성능 평가
print("catboost")
get_clf_eval(y_test, catboost_pred)

catboost
오차행렬
[[1163    0]
 [   5    0]]
정확도: 0.9957, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000
ROC 스코어: 0.5000, PR score : 0.0043
